### Credenciais de Acesso Kaggle

In [1]:
{"username":"jcss03","key":"caf02cc87bdafff295f9eb6897072757"}

{'username': 'jcss03', 'key': 'caf02cc87bdafff295f9eb6897072757'}

### Imports

In [2]:
!pip install opendatasets
!pip install pandas

import opendatasets as od
import pandas
# utilities
import re
import numpy as np
import pandas as pd
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import re

from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Download Dataset

In [3]:
# download data from kaggle
od.download("https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis/download?datasetVersionNumber=2")

Skipping, found downloaded files in "./twitter-entity-sentiment-analysis" (use force=True to force download)


In [4]:
# caminho para o dataset
!ls ./twitter-entity-sentiment-analysis

twitter_training.csv  twitter_validation.csv


### Acesso ao Drive

In [5]:
# Acesso a pasta do drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
# exemplo
!ls /content/drive/MyDrive/'Trabalho de Graduação'

'Apresentação sem título.gslides'  'Proposta de TG'
 Fontes.gdoc			    scripts
'Pesquisas - Links.gdoc'	   'Trabalho de Graduação 2022'


### Loading Datasets
* Train
* Test

In [7]:
dataset_train = pd.read_csv('./twitter-entity-sentiment-analysis/twitter_training.csv',names=['Tweet_ID','entity','sentiment','Tweet content'],header=None)
dataset_validation = pd.read_csv('./twitter-entity-sentiment-analysis/twitter_validation.csv',names=['Tweet_ID','entity','sentiment','Tweet content'],header=None)

In [8]:
print('Exemplo do dataset')
dataset_train.sample(5)

Exemplo do dataset


,Tweet_ID,entity,sentiment,Tweet content
22517,4254,CS-GO,Positive,Crazy - games over 10 games old and now.... Bl...
35080,8026,Microsoft,Positive,The new Microsoft Edge 2 is a beauty
50793,6318,FIFA,Irrelevant,How the fuck does FIFA expect you to win again...
58041,11552,TomClancysRainbowSix,Positive,@Rainbow6Game I feared getting shot through my...
63041,7606,MaddenNFL,Negative,@EA @EAMaddenNFL apparently Christian last nam...


#### Encodificar Targets

In [9]:
data = dataset_train[['Tweet content','sentiment']].rename(columns={"Tweet content": "text", "sentiment": "target"})
data.target = pd.Categorical(data.target)

data['code'] = data.target.cat.codes
print('#'*100)
print(f'Targets: {data.target.unique()}')
print(f'Codes: {data.code.unique()}')
print('#'*100)
data.sample(5)

####################################################################################################
Targets: ['Positive', 'Neutral', 'Negative', 'Irrelevant']
Categories (4, object): ['Irrelevant', 'Negative', 'Neutral', 'Positive']
Codes: [3 2 1 0]
####################################################################################################


,text,target,code
25835,When mom tell y’all I’ve never been this hard ...,Positive,3
41265,A ban targeting Battlefield 4 player accounts ...,Irrelevant,0
13643,@NBA2K this game mean suck fucking TF A PURE S...,Negative,1
44564,"Trippin, Verizon",Negative,1
40377,NaN,Negative,1


#### Remover valores Null

In [10]:
print('#'*100)
qtd_null_values = np.sum(data.isnull().any(axis=1))
print(f'Shape dos dados: {data.shape}')
print(f'Number null values = {qtd_null_values}')

# dropna values
data = data.dropna().reset_index(drop=True)

print('#'*100)
qtd_null_values = np.sum(data.isnull().any(axis=1))
print(f'Shape dos dados: {data.shape}')
print(f'Number null values = {qtd_null_values}')

####################################################################################################
Shape dos dados: (74682, 3)
Number null values = 686
####################################################################################################
Shape dos dados: (73996, 3)
Number null values = 0


### Processing Text
* stopwords
* punctuation
* Stemming(reducing the words to their derived stems)
* Lemmatization(reducing the derived words to their root form known as lemma)
* removing unnecessary characters
* lower case text to better generalization

In [11]:
# recebe texto e processa
def processing_text(text,flag_punctuation=False,flag_stopwords=False,flag_stem=False,flag_lemma=False):

  ##############################################################################
  # funções

  # maketrans explained - 'https://www.w3schools.com/python/ref_string_maketrans.asp'
  def cleaning_punctuations(text):
      english_punctuations = string.punctuation
      punctuations_list = english_punctuations

      translator = str.maketrans('', '', punctuations_list)
      return text.translate(translator)
  
  # remover stopwords
  def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split(' ') if word not in stop_words])
    return text

  # removendo chars repetidos
  def cleaning_repeating_char(text):
      return re.sub(r'(.)1+', r'1', text)

  # removing stem
  def stemming(text):
    ps = PorterStemmer()
    text = " ".join([ps.stem(word) for word in text.split(' ')])
    return text

  def lemmatizer(text):
      lm = nltk.WordNetLemmatizer()
      text = " ".join([lm.lemmatize(word) for word in text.split(' ')])
      return text
  ##############################################################################

  # Processing

  # lower all string of text
  text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

  # remover pontuações
  if flag_punctuation:
    text = cleaning_punctuations(text)

  # remover stopwords
  if flag_stopwords:
    text = remove_stopwords(text)

  # remover repetições
  text = cleaning_repeating_char(text)

  # lenma
  if flag_lemma:
    text = lemmatizer(text)

  # stem:
  if flag_stem:
    text = stemming(text)

  return text

In [12]:
example = data.text.iloc[100]
example

'Grounded almost was pretty cool even despite the top tier unfunny writing until we became yet another annoying crafting game. I seriously can’t wait on this shitty trend to die'

In [13]:
clean_text = processing_text(example,flag_punctuation=True,flag_stopwords=True,flag_lemma=True)
clean_text

'grounded almost pretty cool even despite top tier unfunny writing became yet another annoying crafting game seriously cant wait shitty trend die'

In [14]:
# clean text
data['clean_text'] = data['text'].apply(lambda x: processing_text(x,flag_punctuation=True,flag_stopwords=True,flag_lemma=True))

In [15]:
data.sample(5)

,text,target,code,clean_text
61532,You might need a Business Lawyer when your bus...,Irrelevant,0,might need business lawyer business threatened...
46523,"Hey @JustinTrudeau , I read that the entire s...",Negative,1,hey justintrudeau read entire staff staple ...
16326,"A cool game, a shitty body design. Nothing new...",Neutral,2,cool game shitty body design nothing new excep...
32658,Just found out I killed @its_x8 last season. I...,Irrelevant,0,found killed itsx8 last season tag
17359,Fuck buying one for my man I’m gonna buy one f...,Positive,3,fuck buying one man im gonna buy one


### BERT v1

In [16]:
# !pip install simpletransformers


In [17]:
from simpletransformers.language_representation import RepresentationModel
        
sentences = ["Machine Learning and Deep Learning are part of AI", "Data Science will excel in future"] #it should always be a list

# sentences = data['text'].to_list()
model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=True
    )

word_vectors = model.encode_sentences(sentences, combine_strategy=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
word_vectors.shape

(2, 11, 768)

In [19]:
from simpletransformers.language_representation import RepresentationModel
sentences = ["Machine Learning and Deep Learning are part of AI", "Data Science will excel in future"] #it should always be a list
model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=True
    )
sentence_vectors = model.encode_sentences(sentences, combine_strategy="mean")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
sentence_vectors.shape

(2, 768)

In [21]:
len(sentence_vectors[0])

768

In [22]:
ok

NameError: ignored

In [ ]:
# !pip install tensorflow-text

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text


preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [ ]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
text_preprocessed = bert_preprocess_model(data.clean_text.to_list())

In [ ]:
ok

In [ ]:
text_preprocessed = bert_preprocess_model(data.clean_text.to_list())
text_preprocessed.keys()

In [ ]:
bert_model = hub.KerasLayer(encoder_url)
bert_results = bert_model(text_preprocessed)
bert_results.keys()


### BERT


In [ ]:
data = data[data.target != 'Irrelevant']
data = data[['clean_text','code']]
data.shape

In [ ]:
data.columns.values

In [ ]:
data.code.unique() # 3 pos | 2 - neu | 1- neg

In [ ]:
# !pip install bert-for-tf2
# !pip install sentencepiece

In [ ]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
tokenizer.tokenize("don't be so judgmental")

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))


In [ ]:
def bert_tokenize(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [ ]:
data['tokens'] = data.clean_text.apply(lambda x: bert_tokenize(x))
data

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

data['tokens'] = data['clean_text'].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
data.sample(10)

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("target", fontsize=12)
data["target"].reset_index().groupby("target").count().sort_values(by= 
       "index").plot(kind="barh", legend=False, 
        ax=ax).grid(axis='x')
plt.show()

In [ ]:
data = data[data.target != 'Irrelevant']


In [ ]:
print(data.target.unique())
print(data.code.unique())


### Split dataset_train 
* train
* validation

In [ ]:
# from sklearn.model_selection import train_test_split

# # split dataset
# df_train, df_val = train_test_split(data[['code','clean_text']], test_size=0.3)
# ## get target
# target_train = df_train["code"]
# target_val = df_val["code"]

### GloVe

In [ ]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [ ]:
import numpy as np

words = dict()

def add_to_dict(d, filename):
  with open(filename, 'r') as f:
    for line in f.readlines():
      line = line.split(' ')

      try:
        d[line[0]] = np.array(line[1:], dtype=float)
      except:
        continue

add_to_dict(words, 'glove.6B.50d.txt')

In [ ]:
def glove_embedding(tokens,word_dict=words):
  
  vectors = []
  for token in tokens:
    if token not in word_dict:
      continue
    
    token_vector = word_dict[token]
    vectors.append(token_vector)
  
  return np.array(vectors, dtype=float)
#exemplo
glove_embedding(data['tokens'].iloc[0]).shape

In [ ]:
df_train, df_val = train_test_split(data, test_size=0.3)

In [ ]:
def df_to_X_y(dff):
  y = dff['code'].to_numpy().astype(int)

  all_word_vector_sequences = []

  for token in dff['tokens']:
    message_as_vector_seq = glove_embedding(token)
    
    if message_as_vector_seq.shape[0] == 0:
      message_as_vector_seq = np.zeros(shape=(1, 50))

    all_word_vector_sequences.append(message_as_vector_seq)
  
  return all_word_vector_sequences, y

In [ ]:
X_train, y_train = df_to_X_y(df_train)

In [ ]:
print(len(X_train), len(X_train[0]))
print(len(X_train), len(X_train[65]))

In [ ]:
sequence_lengths = []

for i in range(len(X_train)):
  sequence_lengths.append(len(X_train[i]))

import matplotlib.pyplot as plt

plt.hist(sequence_lengths)

In [ ]:
pd.Series(sequence_lengths).describe()

In [ ]:
max_lenght = pd.Series(sequence_lengths).max() + 1

In [ ]:
from copy import deepcopy

def pad_X(X, desired_sequence_length=max_lenght):
  X_copy = deepcopy(X)

  for i, x in enumerate(X):
    x_seq_len = x.shape[0]
    sequence_length_difference = desired_sequence_length - x_seq_len
    
    pad = np.zeros(shape=(sequence_length_difference, 50))

    X_copy[i] = np.concatenate([x, pad])
  
  return np.array(X_copy).astype(float)

In [ ]:
X_train = pad_X(X_train)

In [ ]:
print(len(X_train), len(X_train[0]), X_train.shape)
print(len(X_train), len(X_train[65]), X_train.shape)

In [ ]:
y_train.shape

In [ ]:
X_val, y_val = df_to_X_y(df_val)
X_val = pad_X(X_val)

print(X_val.shape, y_val.shape)

### Model


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential([])
model.add(layers.Input(shape=(max_lenght, 50)))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
!ls /content/drive/MyDrive/'Trabalho de Graduação'/scripts/model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import BinaryCrossentropy


cp = ModelCheckpoint('/content/drive/MyDrive/''Trabalho de Graduação''/scripts/model/', save_best_only=True)
# loss='sparse_categorical_crossentropy'
model.compile(loss=BinaryCrossentropy,optimizer=Adam(learning_rate=0.0001),metrics=['accuracy', AUC(name='auc')])


In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, callbacks=[cp])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Bidirectional,Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

In [ ]:
model_glove = Sequential()
model_glove.add(Embedding(len(words), output_dim=100, input_length=max_lenght, trainable=False))
#model_glove.add(Dropout(0.2))
#model_glove.add(Conv1D(64, 2, activation='relu')) 
model_glove.add(LSTM(64))
model_glove.add(Dense(3, activation='softmax'))

model_glove.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model_glove.fit(X_train, y_train, validation_data=(X_val,y_val), epochs = 2,verbose =1)
